In [10]:
# Install if needed:
# !pip install SPARQLWrapper pandas openpyxl

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

BASE = "http://127.0.0.1:5000/sparql"

# Map ontology keys → output file stems
TARGETS = {
    "bfo":  "ontology bfo-definitions.xlsx",
    "ies":  "ontology ies-definitions.xlsx",
    "qudt": "ontology qudt-definitions.xlsx",
    "ccom": "ontology ccom-definitions.xlsx",
    "ccot": "ontology ccot-definitions.xlsx",
    "to":   "ontology to-definitions.xlsx",
}

DEFINITIONS_QUERY = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX qudt: <http://qudt.org/schema/qudt#>
PREFIX ies:  <http://ies.data.gov.uk/ontology/ies4#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>

SELECT ?iri ?label ?definition
WHERE {
  ?iri a ?classType .
  VALUES ?classType { owl:Class rdfs:Class }
  OPTIONAL { ?iri rdfs:label ?label }
  OPTIONAL { ?iri skos:definition ?definition }
  OPTIONAL { ?iri qudt:description ?qudt_description }
  OPTIONAL { ?iri dc:description ?dc_description }
  OPTIONAL { ?iri rdfs:comment ?rdfs_comment }
  BIND(
    IF(STRSTARTS(STR(?iri), "http://qudt.org/schema/qudt"),
       ?qudt_description,
       ?definition) AS ?definition
  )
  BIND(
    IF(STRSTARTS(STR(?iri), "http://qudt.org/schema/qudt"),
       ?dc_description,
       ?definition) AS ?definition
  )
  BIND(
    IF(STRSTARTS(STR(?iri), "http://ies.data.gov.uk/ontology/ies4"),
       ?rdfs_comment,
       ?definition) AS ?definition
  )
  FILTER NOT EXISTS { ?iri owl:deprecated true }

}
ORDER BY ?label
"""

def run_select(endpoint_url: str, query: str) -> pd.DataFrame:
    sp = SPARQLWrapper(endpoint_url)
    sp.setQuery(query)
    sp.setReturnFormat(JSON)
    data = sp.query().convert()
    rows = []
    for b in data["results"]["bindings"]:
        iri = b.get("iri", {}).get("value")
        label = b.get("label", {}).get("value")
        definition = b.get("definition", {}).get("value")
        rows.append({"iri": iri, "label": label, "definition": definition})
    return pd.DataFrame(rows, columns=["iri", "label", "definition"])

for key, outfile in TARGETS.items():
    endpoint = f"{BASE}/{key}"
    print(f"Querying {endpoint} …")
    df = run_select(endpoint, DEFINITIONS_QUERY)
    # Save to Excel
    df.to_excel(outfile, index=False)
    print(f"Saved {len(df)} rows to {outfile}")


Querying http://127.0.0.1:5000/sparql/bfo …
Saved 84 rows to ontology bfo-definitions.xlsx
Querying http://127.0.0.1:5000/sparql/ies …
Saved 510 rows to ontology ies-definitions.xlsx
Querying http://127.0.0.1:5000/sparql/qudt …
Saved 175 rows to ontology qudt-definitions.xlsx
Querying http://127.0.0.1:5000/sparql/ccom …
Saved 28 rows to ontology ccom-definitions.xlsx
Querying http://127.0.0.1:5000/sparql/ccot …
Saved 33 rows to ontology ccot-definitions.xlsx
Querying http://127.0.0.1:5000/sparql/to …
Saved 75 rows to ontology to-definitions.xlsx
